# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

%nuclio cmd pip install textblob
%nuclio env TO_LANG=fr
%nuclio config spec.build.baseImage = "python:3.7-buster"

In [3]:
from textblob import TextBlob
import os

def handler(context, event):
    context.logger.info('This is an NLP example! ')

    # process and correct the text
    blob = TextBlob(event.body.decode('utf-8'))
    corrected = blob.correct()

    # debug print the text before and after correction
    context.logger.info_with("Corrected text", corrected=str(corrected), orig=str(blob))

    # calculate sentiments
    context.logger.info_with("Sentiment",
                             polarity=str(corrected.sentiment.polarity),
                             subjectivity=str(corrected.sentiment.subjectivity))

    # read target language from environment and return translated text
    lang = os.getenv('TO_LANG','fr')
    return str(corrected.translate(to=lang))

In [4]:
# nuclio: ignore
event = nuclio.Event(body=b'good morninng')
handler(context, event)

Python> 2019-03-14 14:38:16,597 [info] This is an NLP example! 
Python> 2019-03-14 14:38:16,659 [info] Corrected text: {'corrected': 'good morning', 'orig': 'good morninng'}
Python> 2019-03-14 14:38:16,715 [info] Sentiment: {'polarity': '0.7', 'subjectivity': '0.6000000000000001'}


'Bonjour'

In [5]:
%nuclio deploy -n nlp -p ai

%nuclio: ['deploy', '-n', 'nlp', '-p', 'ai', '-c', '/User/nlp-example.ipynb']
%nuclio: [nuclio.deploy] 2019-03-14 14:38:29,034 (info) Building processor image
%nuclio: [nuclio.deploy] 2019-03-14 14:38:31,060 (info) Pushing image
%nuclio: [nuclio.deploy] 2019-03-14 14:38:31,060 (info) Build complete
%nuclio: [nuclio.deploy] 2019-03-14 14:38:35,090 (info) Function deploy complete
%nuclio: [nuclio.deploy] 2019-03-14 14:38:35,129 done updating nlp, function address: 3.122.204.208:32182
%nuclio: function deployed


In [6]:
!curl -X POST -d "how are you" 3.122.204.208:32182

Comment vas-tu